# Financial Guidance Insights From Earnings Press Release
## Using Snowflake Cortex AI COMPLETE Structured Outputs to Extract, Transform, and Automate Financial Guidance

### Create Stage in Snowflake to Hold the Earnings Press Release PDF Files.

In [ ]:
CREATE STAGE COMPANY_EARNINGS_PRESS_RELEASES_STG 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

## Instructions to upload the Earnings Press Release PDF files into the stage.

- #### You can download the press release PDF files stored as a zip archive from [Github here](https://github.com/rrprasan/Finance/tree/main/Snowflake/Notebooks/Company_Financials/Financial_Guidance_Extraction_From_Earnings_Press_Release).
- #### Unzip the files in your local drive. 
- #### Copy the PDF files into the Snowflake internal stage - COMPANY_EARNINGS_PRESS_RELEASES_STG.

- #### Follow the [following steps](https://docs.snowflake.com/en/user-guide/data-load-local-file-system-stage-ui#upload-files-onto-a-named-internal-stage) in Snowsight to upload the files into the stage.  
#### 1. Sign in to Snowsight.
#### 2. Select Data » Add Data.
#### 3. On the Add Data page, select Load files into a Stage.
#### 4. In the Upload Your Files dialog that appears, select the files that you want to upload. You can upload multiple files at the same time.
#### 5. Select the database schema in which you created the stage, then select the stage.
#### 6. Optionally, select or create a path where you want to save your files within the stage.
#### 7. Select Upload.

#### The following press release PDF files are in the zipped archive:

- ADBE_Q4_FY_2024_Earnings_Press_Release.pdf
- AMZN_Q3_FY_2024_Earnings_Press_Release.pdf
- CRM_Q3_FY_2025_Earnings_Press_Release.pdf
- CSCO_Q1_FY_2025_Earnings_Press_Release.pdf
- GOOGL_Q3_FY_2024_Earnings_Press_Release.pdf
- MSFT_Q1_FY_2025_Earnings_Press_Release.pdf
- NOW_Q3_FY_2024_Earnings_Press_Release.pdf
- ORCL_Q2_FY_2025_Earnings_Press_Release.pdf
- PANW_Q1_FY_2025_Earnings_Press_Release.pdf
- PG_Q1_FY_2025_Earnings_Press_Release.pdf

### After the upload is complete, list the files in the stage.  

In [ ]:
LIST @COMPANY_EARNINGS_PRESS_RELEASES_STG;

### CREATE A SEQ FOR PRESS_RELEASES_TBL. 
### We will use the Sequence as a unique ID (`PRESS_RELEASE_ID`) in the `COMPANY_PRESS_RELEASES_TBL`.
### This ID is not essential. 
### You can remove it from the table definition below, if you choose.  

In [ ]:
--****************
-- CREATE A SEQ FOR PRESS_RELEASES_TBL. 
-- We will use the Sequence as a ID in the COMPANY_PRESS_RELEASES_TBL.
-- This ID is not essential. 
-- You can remove it from the table definition below. 
--****************
CREATE OR REPLACE SEQUENCE PRESS_RELEASE_ID_SEQ;
SELECT PRESS_RELEASE_ID_SEQ.NEXTVAL;

### The Purpose of `COMPANY_PRESS_RELEASES_TBL`
This table, `COMPANY_PRESS_RELEASES_TBL`, serves as a centralized repository for storing the full text of earnings call press releases from various publicly traded companies.

It's designed to capture key details about each press release, including:

- A unique ID - `PRESS_RELEASE_ID` - for each release.
- The company's name - `COMPANY_NAME` - and its stock ticker symbol - `TICKER_SYMBOL`.
- The type of press release - `PRESS_RELEASE_TYPE` - (specifically noting if it's an "earnings" release, but also allowing for "news" or other types).
- The fiscal period - `PRESS_RELEASE_FISCAL_PERIOD` - (like "Q1 FY 2025" or "Full-Year FY 2025") the release pertains to.
- The date the press release - `PRESS_RELEASE_DATE` - was issued.
- Most importantly, the entire content - `PRESS_RELEASE_CONTENT` - of the press release itself.
- In short, it's a dedicated place to store and organize the raw text of these important company announcements, making them easy to retrieve and analyze.

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE COMPANY_PRESS_RELEASES_TBL (
    PRESS_RELEASE_ID NUMBER DEFAULT PRESS_RELEASE_ID_SEQ.NEXTVAL,
    COMPANY_NAME VARCHAR,
    TICKER_SYMBOL VARCHAR(10),
    PRESS_RELEASE_TYPE VARCHAR COMMENT 'PRESS RELEASE TYPE EARNINGS, NEWS',
    PRESS_RELEASE_FISCAL_PERIOD VARCHAR COMMENT 'SHOULD FOLLOW THIS FORMAT: Q1 FY 2025 OR FOR ANNUAL EARNINGS FULL-YEAR FY 2025',
    PRESS_RELEASE_DATE DATE,
    PRESS_RELEASE_CONTENT VARCHAR
);

# PARSE DOCUMENT
#### [PARSE_DOCUMENT](https://docs.snowflake.com/en/user-guide/snowflake-cortex/parse-document)

The PARSE_DOCUMENT function is a Cortex AI **task-specific function** that gives you the ability to **extract text** or **layout** from documents stored in an internal or external stage. PARSE_DOCUMENT combines powerful Optical Character Recognition (OCR) capabilities with machine learning models to identify text content, information stored in tables, and the structural elements of PDF documents. You can use the PARSE_DOCUMENT function to extract text and document layout to build information retrieval systems on large archives of business documents, and to load the extracted information into structured Snowflake tables for use by your applications.

### Parse Adobe's Earnings Press Release PDF Stored in the Stage. 

In [ ]:
-- Parse Adobe's Earnings Press Release 
-- This is a test of the PARSE_DOCUMENT Function. 
-- company_earnings_press_releases_stg/ADBE_Q4_FY_2024_Earnings_Press_Release.pdf
SELECT TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@COMPANY_EARNINGS_PRESS_RELEASES_STG','ADBE_Q4_FY_2024_Earnings_Press_Release.pdf', {'mode': 'LAYOUT'}):content);

### Parse each PDF File and load the contents into the PRESS_RELEASE_CONTENT column.  
### We have the other columns with the metadata information about each PRESS_RELEASE:
- COMPANY_NAME
- TICKER_SYMBOL
- PRESS_RELEASE_TYPE
- PRESS_RELEASE_FISCAL_PERIOD
- PRESS_RELEASE_DATE
### Process Amazon's Earnings Press Release. 

In [ ]:
INSERT INTO COMPANY_PRESS_RELEASES_TBL 
    (COMPANY_NAME, 
     TICKER_SYMBOL, 
     PRESS_RELEASE_TYPE, 
     PRESS_RELEASE_FISCAL_PERIOD, 
     PRESS_RELEASE_DATE, 
     PRESS_RELEASE_CONTENT)
SELECT 'AMAZON.COM, INC.', 
        'AMZN', 
        'EARNINGS',
        'Q3 FY 2024', 
        '2024-12-03',       
TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@COMPANY_EARNINGS_PRESS_RELEASES_STG','AMZN_Q3_FY_2024_Earnings_Press_Release.pdf', {'mode': 'LAYOUT'}):content);

### Process Adobe's Earnings Press Release

In [ ]:
INSERT INTO COMPANY_PRESS_RELEASES_TBL 
    (COMPANY_NAME, 
     TICKER_SYMBOL, 
     PRESS_RELEASE_TYPE, 
     PRESS_RELEASE_FISCAL_PERIOD, 
     PRESS_RELEASE_DATE, 
     PRESS_RELEASE_CONTENT)
SELECT 'ADOBE INC.', 
        'ADBE', 
        'EARNINGS',
        'Q4 FY 2024', 
        '2024-12-11',       
TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@COMPANY_EARNINGS_PRESS_RELEASES_STG','ADBE_Q4_FY_2024_Earnings_Press_Release.pdf', {'mode': 'LAYOUT'}):content);

### Process Salesforce's Earnings Press Release

In [ ]:
INSERT INTO COMPANY_PRESS_RELEASES_TBL 
    (COMPANY_NAME, 
     TICKER_SYMBOL, 
     PRESS_RELEASE_TYPE, 
     PRESS_RELEASE_FISCAL_PERIOD, 
     PRESS_RELEASE_DATE, 
     PRESS_RELEASE_CONTENT)
SELECT 'Salesforce, Inc.', 
        'CRM', 
        'EARNINGS',
        'Q3 FY 2025', 
        '2024-12-03',       
TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@COMPANY_EARNINGS_PRESS_RELEASES_STG','CRM_Q3_FY_2025_Earnings_Press_Release.pdf', {'mode': 'LAYOUT'}):content);

### Process Cisco Systems' Earnings Press Release

In [ ]:
INSERT INTO COMPANY_PRESS_RELEASES_TBL 
    (COMPANY_NAME, 
     TICKER_SYMBOL, 
     PRESS_RELEASE_TYPE, 
     PRESS_RELEASE_FISCAL_PERIOD, 
     PRESS_RELEASE_DATE, 
     PRESS_RELEASE_CONTENT)
SELECT 'CISCO SYSTEMS, INC.', 
        'CSCO', 
        'EARNINGS',
        'Q1 FY 2025', 
        '2024-11-13',       
TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@COMPANY_EARNINGS_PRESS_RELEASES_STG','CSCO_Q1_FY_2025_Earnings_Press_Release.pdf', {'mode': 'LAYOUT'}):content);

### Process Google's Earnings Press Release

In [ ]:
INSERT INTO COMPANY_PRESS_RELEASES_TBL 
    (COMPANY_NAME, 
     TICKER_SYMBOL, 
     PRESS_RELEASE_TYPE, 
     PRESS_RELEASE_FISCAL_PERIOD, 
     PRESS_RELEASE_DATE, 
     PRESS_RELEASE_CONTENT)
SELECT 'Alphabet, Inc.', 
        'GOOGL', 
        'EARNINGS',
        'Q3 FY 2024', 
        '2024-10-29',       
TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@COMPANY_EARNINGS_PRESS_RELEASES_STG','GOOGL_Q3_FY_2024_Earnings_Press_Release.pdf', {'mode': 'LAYOUT'}):content);

### Process Microsoft's Earnings Press Release

In [ ]:
INSERT INTO COMPANY_PRESS_RELEASES_TBL 
    (COMPANY_NAME, 
     TICKER_SYMBOL, 
     PRESS_RELEASE_TYPE, 
     PRESS_RELEASE_FISCAL_PERIOD, 
     PRESS_RELEASE_DATE, 
     PRESS_RELEASE_CONTENT)
SELECT 'Microsoft Corporation', 
        'MSFT', 
        'EARNINGS',
        'Q1 FY 2025', 
        '2024-10-30',       
TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@COMPANY_EARNINGS_PRESS_RELEASES_STG','MSFT_Q1_FY_2025_Earnings_Press_Release.pdf', {'mode': 'LAYOUT'}):content);

### Process ServiceNow's Earnings Press Release

In [ ]:
INSERT INTO COMPANY_PRESS_RELEASES_TBL 
    (COMPANY_NAME, 
     TICKER_SYMBOL, 
     PRESS_RELEASE_TYPE, 
     PRESS_RELEASE_FISCAL_PERIOD, 
     PRESS_RELEASE_DATE, 
     PRESS_RELEASE_CONTENT)
SELECT 'SERVICENOW, INC.', 
        'NOW', 
        'EARNINGS',
        'Q3 FY 2024', 
        '2024-10-23',       
TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@COMPANY_EARNINGS_PRESS_RELEASES_STG','NOW_Q3_FY_2024_Earnings_Press_Release.pdf',{'mode': 'LAYOUT'}):content);

### Process Oracle's Earnings Press Release

In [ ]:
INSERT INTO COMPANY_PRESS_RELEASES_TBL 
    (COMPANY_NAME, 
     TICKER_SYMBOL, 
     PRESS_RELEASE_TYPE, 
     PRESS_RELEASE_FISCAL_PERIOD, 
     PRESS_RELEASE_DATE, 
     PRESS_RELEASE_CONTENT)
SELECT 'Oracle Corporation', 
        'ORCL', 
        'EARNINGS',
        'Q2 FY 2025', 
        '2024-12-09',       
TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@COMPANY_EARNINGS_PRESS_RELEASES_STG','ORCL_Q2_FY_2025_Earnings_Press_Release.pdf',{'mode': 'LAYOUT'}):content);

### Process Palo Alto Networks' Earnings Press Release

In [ ]:
INSERT INTO COMPANY_PRESS_RELEASES_TBL 
    (COMPANY_NAME, 
     TICKER_SYMBOL, 
     PRESS_RELEASE_TYPE, 
     PRESS_RELEASE_FISCAL_PERIOD, 
     PRESS_RELEASE_DATE, 
     PRESS_RELEASE_CONTENT)
SELECT 'PALO ALTO NETWORKS, INC', 
        'PANW', 
        'EARNINGS',
        'Q1 FY 2025', 
        '2024-11-20',       
TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@COMPANY_EARNINGS_PRESS_RELEASES_STG','PANW_Q1_FY_2025_Earnings_Press_Release.pdf',{'mode': 'LAYOUT'}):content);

### Process Procter & Gamble's Earnings Press Release

In [ ]:
INSERT INTO COMPANY_PRESS_RELEASES_TBL 
    (COMPANY_NAME, 
     TICKER_SYMBOL, 
     PRESS_RELEASE_TYPE, 
     PRESS_RELEASE_FISCAL_PERIOD, 
     PRESS_RELEASE_DATE, 
     PRESS_RELEASE_CONTENT)
SELECT 'Procter & Gamble', 
        'PG', 
        'EARNINGS',
        'Q1 FY 2025', 
        '2024-10-18',       
TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@COMPANY_EARNINGS_PRESS_RELEASES_STG','PG_Q1_FY_2025_Earnings_Press_Release.pdf', {'mode': 'LAYOUT'}):content);

# Efficiently Analyzing Earnings Press Releases with Cortex AI
- ## Direct Application & Its Drawbacks: 
    - You can directly use the Cortex COMPLETE function on an entire earnings press release to get answers based on your prompt. However, this approach is typically not the most efficient way to gain targeted insights from the document.
- ## A More Effective Strategy: 
    - For greater efficiency and precision, it is preferable to feed the COMPLETE function only the most relevant chunks of text and data from the press release that directly address your prompt.
- ## The Value of Cortex Search: 
    - This is where Cortex Search provides significant value. It allows you to use your prompt to first pinpoint and retrieve these pertinent sections from the document. 
    - Supplying this curated input to the COMPLETE function makes the insight generation process more focused and efficient.

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', [
        {
        'role': 'user',
        'content': 
        'Extract the following from the Press_Release_Content: 1. Quarterly Revenue Target: Identify the reported forecasted, targeted, or guidance quarterly revenue. If a range is provided, return the lower and upper bounds separately.
2.  Annual Revenue Target: Identify the forecasted, targeted, or guidance total annual revenue. If a range is provided, return the lower and upper bounds separately.
3. Missing Data: If any quarterly or annual target revenue/sales data is unavailable, state \'NOT_FOUND_OR_AVAILABLE\'.
4. Forecast Type:
For both quarterly and annual revenue forecasts, specify if the forecast is \'dollar_forecast\' or \'percentage_forecast\'.
' || PRESS_RELEASE_CONTENT
            }
    ],
    {
        'temperature': 0,
        'max_tokens': 8192,
        'response_format':{
            'type':'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'revenue_targets': {
                        'type': 'array',
                        'items': {
                            'type': 'object',
                            'properties': {
                                'lower_quarterly_revenue_target': {'type': 'string'},
                                'upper_quarterly_revenue_target': {'type': 'string'},
                                'lower_annual_revenue_target': {'type': 'string'},
                                'upper_annual_revenue_target': {'type': 'string'},
                                'quarterly_forecast_type': {'type': 'string'},
                                'annual_forecast_type': {'type': 'string'}
                            },
                            'required': []
                        }
                    }
                }
            }
            }
    }
) Revenue_Forecast 
FROM 
    DEMODB.EQUITY_RESEARCH.COMPANY_PRESS_RELEASES_TBL 
WHERE 
    TICKER_SYMBOL = 'NOW';

## Output from the SELECT SQL With COMPLETE Function.
```JSON
{
  "created": 1748575481,
  "model": "claude-3-5-sonnet",
  "structured_output": [
    {
      "raw_message": {
        "revenue_targets": [
          {
            "annual_forecast_type": "dollar_forecast",
            "lower_annual_revenue_target": "10655",
            "lower_quarterly_revenue_target": "2875",
            "quarterly_forecast_type": "dollar_forecast",
            "upper_annual_revenue_target": "10660",
            "upper_quarterly_revenue_target": "2880"
          }
        ]
      },
      "type": "json"
    }
  ],
  "usage": {
    "completion_tokens": 33,
    "prompt_tokens": 9503,
    "total_tokens": 9536
  }
}
```

## Why Use Cortex Search with the COMPLETE Function? 
- ### Direct COMPLETE Function Use: 
    - Applying the Cortex AI COMPLETE function directly to an entire press release PDF is an option, but it can be inefficient and costly.
- ### Tailored Input for Efficiency: 
    - For better results and cost-effectiveness, it's advisable to tailor the input for the COMPLETE function based on the specific question (prompt) you're aiming to answer.
- ### Leveraging Cortex Search: 
    - Cortex Search facilitates this by allowing you to use your prompt to first identify and extract the most relevant text segments from the press release.
- ### Optimized Workflow: 
    - Feeding these targeted search results into the COMPLETE function creates a more efficient and focused information extraction process. 

## Create [Cortex Search Service](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview) 
Cortex Search enables low-latency, high-quality “fuzzy” search over your Snowflake data. Cortex Search powers a broad array of search experiences for Snowflake users including Retrieval Augmented Generation (RAG) applications leveraging Large Language Models (LLMs).

Cortex Search gets you up and running with a hybrid (vector and keyword) search engine on your text data in minutes, without having to worry about embedding, infrastructure maintenance, search quality parameter tuning, or ongoing index refreshes. This means you can spend less time on infrastructure and search quality tuning, and more time developing high-quality chat and search experiences using your data. Check out the Cortex Search tutorials for step-by-step instructions on using Cortex Search to power AI chat and search applications.

- ### We split the text from the press release documents to create text chunks.  
- ### Cortex Search service creates vectors on the text chunks. 
- ### Use the Snowflake Cortex [SPLIT_TEXT_RECURSIVE_CHARACTER](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex) to create chunks from each press release.  
- ### Store the chunks along with the meta data about each press release into a new "chunks" table - COMPANY_PRESS_RELEASES_CHUNKS_TBL

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE COMPANY_PRESS_RELEASES_CHUNKS_TBL 
AS
SELECT
   COMPANY_NAME, 
     TICKER_SYMBOL, 
     PRESS_RELEASE_TYPE, 
     PRESS_RELEASE_FISCAL_PERIOD, 
     PRESS_RELEASE_DATE, 
     PRESS_RELEASE_CONTENT,
     TO_VARCHAR(CHUNK_TXT.VALUE) AS PRESS_RELEASE_CHUNK
FROM
   COMPANY_PRESS_RELEASES_TBL,
   LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
      PRESS_RELEASE_CONTENT,
      'none',
      300,
      75
   )) CHUNK_TXT;

In [ ]:
CREATE CORTEX SEARCH SERVICE COMPANY_PRESS_RELEASES_SEARCH_SVC
    ON PRESS_RELEASE_CHUNK
    ATTRIBUTES COMPANY_NAME, TICKER_SYMBOL, PRESS_RELEASE_TYPE
    WAREHOUSE = COMPUTE_WH
    TARGET_LAG = '1 hour'
    AS (
        SELECT 
        *
        FROM COMPANY_PRESS_RELEASES_CHUNKS_TBL
    );

### Test the Cortex Search Service by using the Snowflake Cortex [SEARCH_PREVIEW](https://docs.snowflake.com/en/sql-reference/functions/search_preview-snowflake-cortex) SQL function.  

In [ ]:
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'DEMODB.EQUITY_RESEARCH.COMPANY_PRESS_RELEASES_SEARCH_SVC',
      '{
        "query": "Summarize the earnings press release",
        "columns":[
            "PRESS_RELEASE_CHUNK"
        ],
        "filter": {"@eq": {"TICKER_SYMBOL": "ADBE"} },
        "limit":5
      }'
  )
)['results'] as results;

In [ ]:
# Import python packages
import streamlit as st
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session

In [ ]:
llm_option = st.selectbox(
    'Select a LLM:',
    ('claude-3-5-sonnet', 'llama3.2-1b', 'llama3.2-3b',
     'llama3.1-8b', 'llama3.1-70b', 'llama3.1-405b',
     'snowflake-arctic', 'reka-core', 'reka-flash',
     'mistral-large2', 'mixtral-8x7b', 'mistral-7b',
     'jamba-instruct', 'jamba-1.5-mini', 'jamba-1.5-large',
     'gemma-7b'))
str_selected_llm = f"""You Selected {llm_option} LLM Model. This model will be used in the SNOWFLAKE.CORTEX.COMPLETE function."""
st.write(str_selected_llm)

In [ ]:
session = get_active_session()
root = Root(session)
selected_company = st.selectbox('Select a Company:', 
                session.sql("SELECT DISTINCT COMPANY_NAME FROM COMPANY_PRESS_RELEASES_TBL").collect())

st.write(selected_company)

In [ ]:
database_name = 'DEMODB'
schema_name = 'EQUITY_RESEARCH'
cortex_search_service_name = 'COMPANY_PRESS_RELEASES_SEARCH_SVC'

### The Prompt:
Extract the following from the Press_Release_Content:

-   **Quarterly Revenue Target:** Identify the reported forecasted, targeted, or guidance quarterly revenue. If a range is provided, return the lower and upper bounds separately.
-   **Annual Revenue Target:** Identify the forecasted, targeted, or guidance total annual revenue. If a range is provided, return the lower and upper bounds separately.
-   **Missing Data:** If any quarterly or annual target revenue/sales data is unavailable, state 'NOT_FOUND_OR_AVAILABLE'.
-   **Forecast Type:** For both quarterly and annual revenue forecasts, specify if the forecast is 'dollar_forecast' or 'percentage_forecast'.

In [ ]:
prompt_str = f"""Extract the following from the Press_Release_Content:
-   **Quarterly Revenue Target:** Identify the reported forecasted, targeted, or guidance quarterly revenue. If a range is provided, return the lower and upper bounds separately.
-   **Annual Revenue Target:** Identify the forecasted, targeted, or guidance total annual revenue. If a range is provided, return the lower and upper bounds separately.
-   **Missing Data:** If any quarterly or annual target revenue/sales data is unavailable, state 'NOT_FOUND_OR_AVAILABLE'.
-   **Forecast Type:** For both quarterly and annual revenue forecasts, specify if the forecast is 'dollar_forecast' or 'percentage_forecast'.
"""

In [ ]:
cortex_search_service = (
        root.databases[database_name]
        .schemas[schema_name]
        .cortex_search_services[cortex_search_service_name]
    )
context_documents = cortex_search_service.search(
         prompt_str, 
         columns=['PRESS_RELEASE_CHUNK'], 
         filter={"@eq": {"COMPANY_NAME": selected_company} },
         limit=12
    )
results = context_documents.results

In [ ]:
# Define the HTML code for the sub-heading
html_code_llm_response = """
                        <h2>Here are the results returned by Cortex Search:</h2>
                        """

# Display the HTML code using st.write()
st.write(html_code_llm_response, unsafe_allow_html=True) 
st.write(results)

### Concatenate the Search Results to Feed it to the Cortex AI COMPLETE LLM SQL Function.

In [ ]:
search_results_str = ""
for i, r in enumerate(results):
    # st.write(r)
    # context_str += f"Context document {i+1}: {r['PRESS_RELEASE_CHUNK']} \n" + "\n"
    search_results_str += f"{r['PRESS_RELEASE_CHUNK']} \n"
    # st.write(context_str)
st.write(search_results_str)

In [ ]:
import json
contentStr = f"""Extract the following from the Press_Release_Content:
-   **Quarterly Revenue Target:** Identify the reported forecasted, targeted, or guidance quarterly revenue. If a range is provided, return the lower and upper bounds separately.
-   **Annual Revenue Target:** Identify the forecasted, targeted, or guidance total annual revenue. If a range is provided, return the lower and upper bounds separately.
-   **Missing Data:** If any quarterly or annual target revenue/sales data is unavailable, state 'NOT_FOUND_OR_AVAILABLE'.
-   **Forecast Type:** For both quarterly and annual revenue forecasts, specify if the forecast is 'dollar_forecast' or 'percentage_forecast'. {search_results_str}""" 
# Your provided structure as a Python list of dictionaries
# Note: I'm replacing the SQL-like '|| PRESS_RELEASE_CONTENT' with a placeholder
# for now, as that's a SQL concatenation, not Python.
# In a real scenario, you'd insert the actual content here.
data_structure = [
    {
        'role': 'user',
        'content': contentStr 
    },
    {
        'temperature': 0,
        'max_tokens': 8192,
        'response_format': {
            'type': 'json',
            'schema': {
                'type': 'object',
                'properties': {
                    'revenue_targets': {
                        'type': 'array',
                        'items': {
                            'type': 'object',
                            'properties': {
                                'lower_quarterly_revenue_target': {'type': 'string'},
                                'upper_quarterly_revenue_target': {'type': 'string'},
                                'lower_annual_revenue_target': {'type': 'string'},
                                'upper_annual_revenue_target': {'type': 'string'},
                                'quarterly_forecast_type': {'type': 'string'},
                                'annual_forecast_type': {'type': 'string'}
                            },
                            'required': []
                        }
                    }
                }
            }
        }
    }
]

# Convert the Python data structure to a JSON string
json_string = json.dumps(data_structure, indent=4) # indent for pretty printing

print(json_string)

# Example of how you might dynamically insert the press release content
press_release_content_variable = "This is the actual press release content for the query."
data_structure[0]['content'] = data_structure[0]['content'].replace("[PLACEHOLDER_PRESS_RELEASE_CONTENT]", press_release_content_variable)

# Convert to JSON string again with the updated content
json_string_with_content = json.dumps(data_structure, indent=4)
print("\n--- JSON string with actual content inserted ---")
print(json_string_with_content)

In [ ]:
results = session.sql("SELECT SNOWFLAKE.CORTEX.COMPLETE(?, ?) Revenue_Forecast FROM DEMODB.EQUITY_RESEARCH.COMPANY_PRESS_RELEASES_TBL", (llm_option, json_string_with_content)).collect()[0][0]

# Define the HTML code for the sub-heading
html_code_llm_response = f"""
                            <h2>Here's the Structured output (JSON) of {selected_company} from its Earnings Press Release:</h2>
                          """

# Display the HTML code using st.write()
st.write(html_code_llm_response, unsafe_allow_html=True) 
st.write(results)

## Parse the JSON (Optional) and prepare for inserting the data into a table. 

In [ ]:
import json

# Assuming 'results' is a JSON string
# results = '{"revenue_targets": [100, 200], "other_data": "some_value"}'

try:
    results_dict = json.loads(results)
    revenue_targets_list = results_dict.get("revenue_targets")
    st.write(revenue_targets_list)
    lowerBoundNextQtrRevenue = revenue_targets_list[0]["lower_quarterly_revenue_target"]
    upperBoundNextQtrRevenue = revenue_targets_list[0]["upper_quarterly_revenue_target"]
    qtrlyRevenueForecastType = revenue_targets_list[0]["quarterly_forecast_type"]
    lowerBoundAnnualRevenue = revenue_targets_list[0]["lower_annual_revenue_target"]
    upperBoundAnnualRevenue = revenue_targets_list[0]["upper_annual_revenue_target"]
    annualRevenueForecastType = revenue_targets_list[0]["annual_forecast_type"]
    st.write(f"""Lower Bound Next Quarter Revenue Target: {revenue_targets_list[0]["lower_quarterly_revenue_target"]}""")
    st.write(f"""Upper Bound Next Quarter Revenue Target: {revenue_targets_list[0]["upper_quarterly_revenue_target"]}""")
    st.write(f"""Quarterly Forecast Type Provided by the Company (Dollar or Percentage): {revenue_targets_list[0]["quarterly_forecast_type"]}""")
    st.write(f"""Lower Bound Next Annual Revenue Target: {revenue_targets_list[0]["lower_annual_revenue_target"]}""")
    st.write(f"""Upper Bound Next Annual Revenue Target: {revenue_targets_list[0]["upper_annual_revenue_target"]}""")
    st.write(f"""Annual Forecast Type Provided by the Company (Dollar or Percentage) {revenue_targets_list[0]["annual_forecast_type"]}""")
except json.JSONDecodeError:
    print("Error: 'results' is not a valid JSON string.")
    # Handle the error, perhaps by logging it or setting a default value

## Create a table to store the results obtained from the earnings press release document.  

In [ ]:
CREATE TRANSIENT TABLE COMPANY_FINANCIAL_OUTLOOK_TBL
(
    COMPANY_NAME VARCHAR COMMENT 'Store the name of the company which issued the financial forecast (provided an outlook for revenue or profit or both)',
    LOWER_BOUND_NEXT_QUARTER_REVENUE_TARGET VARCHAR COMMENT 'Store the lower bound of the revenue target for the next quarter provided by the company',
    UPPER_BOUND_NEXT_QUARTER_REVENUE_TARGET VARCHAR,
    TYPE_OF_NEXT_QUARTER_FORECAST VARCHAR COMMENT 'The type of next quarter forecast provided by the company: Dollar or Percentage',
    LOWER_BOUND_ANNUAL_REVENUE_TARGET VARCHAR,
    UPPER_BOUND_ANNUAL_REVENUE_TARGET VARCHAR,
    TYPE_OF_NEXT_ANNUAL_FORECAST VARCHAR COMMENT 'The type of next annual forecast provided by the company: Dollar or Percentage'  
)

In [ ]:
insertSQLStr = f"""insert into 
    COMPANY_FINANCIAL_OUTLOOK_TBL 
VALUES 
    (\'{selected_company}\', 
     \'{lowerBoundNextQtrRevenue}\', 
     \'{upperBoundNextQtrRevenue}\',
     \'{qtrlyRevenueForecastType}\',
     \'{lowerBoundAnnualRevenue}\',
     \'{upperBoundAnnualRevenue}\',
     \'{annualRevenueForecastType}\'
     )"""
st.write(insertSQLStr)
session.sql(insertSQLStr).collect()

# Snowflake = EASY
## Snowflake has made it easy for your extract data and insights from a document and store that in a table or use it to drive any workflow. 


In [ ]:
SELECT * FROM COMPANY_FINANCIAL_OUTLOOK_TBL;